# 맥락 보존 개선된 RAG Chain 생성

스펙과 규격이 잘리지 않도록 개선된 텍스트 분할 방식

In [114]:
import os
import re
from typing import List, Dict, Any
from pathlib import Path
from langchain.docstore.document import Document
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import json

In [115]:
def load_markdown_file(file_path: str) -> str:
    """마크다운 파일을 로드합니다."""
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 마크다운 파일 로드
md_content = load_markdown_file("data/dev_center_guide_touched.md")
print(f"마크다운 파일 크기: {len(md_content)} 문자")
print(f"첫 500자: {md_content[:500]}...")

마크다운 파일 크기: 112832 문자
첫 500자: # 원스토어 인앱 결제 연동 가이드

## 원스토어 In-App SDK
원스토어 인앱결제는 다양한 결제수단과 강력한 보안 그리고 편리하고 안전한 결제 서비스를 제공합니다.

- 최신버전 인앱결제 가이드
[원스토어 인앱결제 API V7 (SDK V21) 안내 및 다운로드](#원스토어-인앱결제-api-v7sdk-v21-연동-안내-및-다운로드)

## 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드
원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다.
보다 강력하고 다양한 기능을 지원하는 최신 버전을 적용해보세요.

- API V4(SDK V16) 이하 버전과는 호환되지 않습니다. 인앱결제 API V4(SDK V16)에 대한 안내 및 다운로드는 여기를 클릭해주세요.
- 현재 판매중인 앱을 대한민국 외 국가/지역으로 배포하기 위해서는 아래 가이드를 참고해주세요
  - [대한민국 외 국가 및 지역 배포를 위한 가이드](https://onestore-de...


In [116]:
def context_preserving_split(md_text: str, chunk_size: int = 800, chunk_overlap: int = 150) -> List[Document]:
    """
    맥락 보존 분할 함수
    - 더 큰 청크 크기로 분할하여 맥락 보존
    - 스펙과 규격이 잘리지 않도록 개선
    - 헤더 기반 분할 우선 적용
    """
    
    # 1단계: 코드 블록 보존하면서 분할
    # 코드 블록을 임시로 마스킹
    code_blocks = {}
    counter = 0
    
    def mask_code_block(match):
        nonlocal counter
        placeholder = f"__CODE_BLOCK_{counter}__"
        code_blocks[placeholder] = match.group(0)
        counter += 1
        return placeholder
    
    # 코드 블록 마스킹
    masked_text = re.sub(r'```[\\s\\S]*?```', mask_code_block, md_text)
    
    # 2단계: 헤더 기반 분할 우선 적용
    documents = []
    current_chunk = ""
    current_header = ""
    
    lines = masked_text.split('\\n')
    i = 0
    
    while i < len(lines):
        line = lines[i]
        
        # 헤더 발견 시 새로운 청크 시작
        if line.strip().startswith('#'):
            # 이전 청크가 있으면 저장
            if current_chunk.strip():
                # 코드 블록 복원
                restored_chunk = current_chunk
                for placeholder, code_block in code_blocks.items():
                    if placeholder in current_chunk:
                        restored_chunk = restored_chunk.replace(placeholder, code_block)
                
                # 키워드 추출
                keywords = extract_keywords(restored_chunk)
                
                doc = Document(
                    page_content=restored_chunk,
                    metadata={
                        "type": "documentation",
                        "source": "dev_center_guide_touched.md",
                        "chunk_idx": len(documents),
                        "chunk_size": chunk_size,
                        "chunk_overlap": chunk_overlap,
                        "header": current_header,
                        "keywords": keywords,
                        "has_code": "```" in restored_chunk,
                        "has_function": any(func in restored_chunk for func in ["()", "function", "API", "launch"]),
                        "split_method": "header_based"
                    }
                )
                documents.append(doc)
            
            # 새로운 헤더로 시작
            current_header = line.strip()
            current_chunk = line + '\\n'
            i += 1
            
            # 헤더 다음 내용들을 수집
            while i < len(lines):
                next_line = lines[i]
                
                # 다음 헤더를 만나면 중단
                if next_line.strip().startswith('#'):
                    break
                
                current_chunk += next_line + '\\n'
                i += 1
                
                # 청크가 너무 커지면 중단 (스펙 보존을 위해)
                if len(current_chunk) > chunk_size * 2:
                    break
        else:
            # 일반 텍스트는 기존 방식으로 처리
            current_chunk += line + '\\n'
            i += 1
    
    # 마지막 청크 처리
    if current_chunk.strip():
        # 코드 블록 복원
        restored_chunk = current_chunk
        for placeholder, code_block in code_blocks.items():
            if placeholder in current_chunk:
                restored_chunk = restored_chunk.replace(placeholder, code_block)
        
        # 키워드 추출
        keywords = extract_keywords(restored_chunk)
        
        doc = Document(
            page_content=restored_chunk,
            metadata={
                "type": "documentation",
                "source": "dev_center_guide_touched.md",
                "chunk_idx": len(documents),
                "chunk_size": chunk_size,
                "chunk_overlap": chunk_overlap,
                "header": current_header,
                "keywords": keywords,
                "has_code": "```" in restored_chunk,
                "has_function": any(func in restored_chunk for func in ["()", "function", "API", "launch"]),
                "split_method": "header_based"
            }
        )
        documents.append(doc)
    
    # 3단계: 큰 청크들을 다시 적절한 크기로 분할 (필요시)
    final_documents = []
    for doc in documents:
        if len(doc.page_content) > chunk_size * 1.5:
            # 큰 청크를 다시 분할
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                separators=[
                    "\\n\\n",  # 빈 줄
                    "\\n",    # 줄바꿈
                    ". ",    # 문장 끝
                    ", ",    # 쉼표
                    " ",     # 공백
                    ""       # 문자 단위
                ],
                length_function=len,
                is_separator_regex=False
            )
            
            sub_chunks = splitter.split_text(doc.page_content)
            for j, sub_chunk in enumerate(sub_chunks):
                sub_doc = Document(
                    page_content=sub_chunk,
                    metadata={
                        **doc.metadata,
                        "chunk_idx": f"{doc.metadata['chunk_idx']}_{j}",
                        "split_method": "recursive_split",
                        "parent_chunk": doc.metadata['chunk_idx']
                    }
                )
                final_documents.append(sub_doc)
        else:
            final_documents.append(doc)
    
    return final_documents

def extract_keywords(text: str) -> List[str]:
    """텍스트에서 키워드를 추출합니다."""
    keywords = []
    
    # 함수명 패턴 (예: launchPurchaseFlow, queryProductDetailAsync)
    function_pattern = r'\\b[a-zA-Z][a-zA-Z0-9]*\\([^)]*\\)'
    functions = re.findall(function_pattern, text)
    keywords.extend(functions)
    
    # API 패턴
    api_pattern = r'\\b[A-Z][A-Z0-9_]*\\b'
    apis = re.findall(api_pattern, text)
    keywords.extend(apis[:10])  # 상위 10개로 증가
    
    # 특수 키워드 (확장)
    special_keywords = [
        "launchPurchaseFlow", "queryProductDetailAsync", "PurchaseClient",
        "IapResult", "PurchaseData", "ProductDetail", "PurchaseFlowParams",
        "PNS", "SNS", "BillingClient", "BillingResult", "SubscriptionNotification",
        "PurchaseFlowParams", "ProrationMode", "AuthManager", "PurchaseManager"
    ]
    
    for keyword in special_keywords:
        if keyword.lower() in text.lower():
            keywords.append(keyword)
    
    return list(set(keywords))  # 중복 제거

# 맥락 보존 분할 실행 (더 큰 청크 크기)
docs_context_preserved = context_preserving_split(md_content, chunk_size=800, chunk_overlap=150)
print(f"맥락 보존 분할 완료: {len(docs_context_preserved)}개 청크")

# launchPurchaseFlow 포함 문서 확인
launch_docs = []
for i, doc in enumerate(docs_context_preserved):
    if 'PNS' in doc.page_content:
        launch_docs.append((i, doc))
        print(f"문서 {i}: {doc.page_content[:200]}...")
        print(f"키워드: {doc.metadata.get('keywords', [])}")
        print(f"분할 방법: {doc.metadata.get('split_method', 'unknown')}")
        print("-" * 50)

print(f"\\n총 {len(launch_docs)}개의 문서에서 'PNS' 발견")

# 청크 크기 통계
chunk_sizes = [len(doc.page_content) for doc in docs_context_preserved]
print(f"평균 청크 크기: {sum(chunk_sizes) / len(chunk_sizes):.0f} 문자")
print(f"최소 청크 크기: {min(chunk_sizes)} 문자")
print(f"최대 청크 크기: {max(chunk_sizes)} 문자")

맥락 보존 분할 완료: 209개 청크
문서 134: . PNS (Push Notification Service) 이용하기
#### 개요 
원스토어는 개발자를 위해 두 가지 Push Notification Service를 제공합니다.
* PNS는 Push Notification Service의 약자입니다.
* PNS는 모바일의 네트워크 연결 불안정성을 보완하기 위해 개발사가 지정한 서버로 개별 사용자의 결제 ...
키워드: ['ProrationMode', 'launchPurchaseFlow', 'PurchaseFlowParams', 'ProductDetail', 'IapResult', 'queryProductDetailAsync', 'SubscriptionNotification', 'PNS', 'SNS', 'PurchaseData', 'PurchaseClient']
분할 방법: recursive_split
--------------------------------------------------
문서 135: . 
정상적인 결제 건인지 Server to Server로 확인하기를 원하신다면 PNS notification을 이용하는 대신, 관련 서버 API로 조회하는 것을 권장합니다.
원스토어는 검증 및 모니터링 목적으로 결제 테스트를 진행할 수 있으며, 해당 테스트 건들도 결제/결제취소 시 동일하게 notification이 발송됩니다. 원스토어가 진행한 결제 테스...
키워드: ['ProrationMode', 'launchPurchaseFlow', 'PurchaseFlowParams', 'ProductDetail', 'IapResult', 'queryProductDetailAsync', 'SubscriptionNotification', 'PNS', 'SNS', 'PurchaseData', 'PurchaseClient']
분할 방법: recursive_split
--------------------------------------------------
문서 153:

In [117]:
def create_context_enhanced_embeddings(docs: List[Document], output_path: str):
    """맥락 보존 향상된 임베딩 생성 및 저장"""
    
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
    
    # FAISS 데이터베이스 생성
    db = FAISS.from_documents(docs, embedding_model)
    
    # 추가 인덱스 생성 (키워드 기반)
    keyword_docs = []
    for doc in docs:
        keywords = doc.metadata.get('keywords', [])
        for keyword in keywords:
            # 키워드별로 별도 문서 생성
            keyword_doc = Document(
                page_content=f"{keyword}: {doc.page_content[:300]}...",
                metadata={
                    **doc.metadata,
                    "keyword": keyword,
                    "is_keyword_doc": True
                }
            )
            keyword_docs.append(keyword_doc)
    
    # 키워드 문서도 임베딩에 추가
    if keyword_docs:
        keyword_db = FAISS.from_documents(keyword_docs, embedding_model)
        # 기존 DB와 병합
        db.merge_from(keyword_db)
    
    # 저장
    db.save_local(output_path)
    print(f"✅ 맥락 보존 임베딩 저장 완료: {output_path}")
    print(f"총 문서 수: {len(docs) + len(keyword_docs)}")
    
    return db

# 맥락 보존 임베딩 생성
output_dir = "models/faiss_vs_rag_iap_v12_context"
os.makedirs(output_dir, exist_ok=True)
context_enhanced_db = create_context_enhanced_embeddings(docs_context_preserved, output_dir)

✅ 맥락 보존 임베딩 저장 완료: models/faiss_vs_rag_iap_v12_context
총 문서 수: 2508


In [119]:
# 맥락 보존 검색 테스트
print("=== 맥락 보존 검색 테스트 ===")

# 임베딩 로드
embedding_model = OllamaEmbeddings(model="exaone3.5:latest")
loaded_db = FAISS.load_local(
    folder_path=output_dir,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

# 다양한 검색 방식 테스트
test_queries = [
    # "launchPurchaseFlow()",
    # "launchPurchaseFlow",
    # "구매 요청 함수",
    # "PurchaseClient",
    # "queryProductDetailAsync",
    # "인앱결제 구매 프로세스",
    # "IapResult",
    # "PurchaseData",
    "PNS"
]

for query in test_queries:
    print(f"\\n🔍 검색 쿼리: {query}")
    
    # 1. 직접 similarity 검색
    results_direct = loaded_db.similarity_search(query, k=3)
    print(f"\\n1. 직접 검색 결과 ({len(results_direct)}개):")
    for i, doc in enumerate(results_direct, 1):
        print(f"  {i}. {doc.page_content}...")
        print(f"     키워드: {doc.metadata.get('keywords', [])}")
        print(f"     분할 방법: {doc.metadata.get('split_method', 'unknown')}")
    
    # 2. MMR 검색
    retriever_mmr = loaded_db.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 3, "fetch_k": 8, "lambda_mult": 0.6}
    )
    results_mmr = retriever_mmr.invoke(query)
    print(f"\\n2. MMR 검색 결과 ({len(results_mmr)}개):")
    for i, doc in enumerate(results_mmr, 1):
        print(f"  {i}. {doc.page_content}...")
        print(f"     키워드: {doc.metadata.get('keywords', [])}")
        print(f"     분할 방법: {doc.metadata.get('split_method', 'unknown')}")
    
    print("=" * 80)

=== 맥락 보존 검색 테스트 ===
\n🔍 검색 쿼리: PNS
\n1. 직접 검색 결과 (3개):
  1. . "...
     키워드: ['ProrationMode', 'launchPurchaseFlow', 'PurchaseFlowParams', 'ProductDetail', 'IapResult', 'queryProductDetailAsync', 'SubscriptionNotification', 'PNS', 'SNS', 'PurchaseData', 'PurchaseClient']
     분할 방법: recursive_split
  2. PurchaseData: . "......
     키워드: ['ProrationMode', 'launchPurchaseFlow', 'PurchaseFlowParams', 'ProductDetail', 'IapResult', 'queryProductDetailAsync', 'SubscriptionNotification', 'PNS', 'SNS', 'PurchaseData', 'PurchaseClient']
     분할 방법: recursive_split
  3. ProductDetail: . "......
     키워드: ['ProrationMode', 'launchPurchaseFlow', 'PurchaseFlowParams', 'ProductDetail', 'IapResult', 'queryProductDetailAsync', 'SubscriptionNotification', 'PNS', 'SNS', 'PurchaseData', 'PurchaseClient']
     분할 방법: recursive_split
\n2. MMR 검색 결과 (3개):
  1. . "...
     키워드: ['ProrationMode', 'launchPurchaseFlow', 'PurchaseFlowParams', 'ProductDetail', 'IapResult', 'queryProductDetailAsync', 'Subscriptio

In [ ]:
# 맥락 보존 RAG 체인 생성
print("=== 맥락 보존 RAG 체인 생성 ===")

# LLM 설정
llm = ChatOllama(model="exaone3.5:latest", temperature=0.3)

# 검색기 설정 (맥락 보존 검색)
retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 12, "lambda_mult": 0.7}
)

# 맥락 보존 프롬프트 템플릿
prompt_template = """
당신은 원스토어 인앱결제 전문가입니다. 주어진 컨텍스트를 바탕으로 질문에 답변해주세요.

답변 시 다음 사항을 고려해주세요:
1. 한국어로 명확하고 이해하기 쉽게 답변하세요
2. 코드 예시가 있다면 포함해주세요
3. 단계별로 설명해주세요
4. 개발자 관점에서 실용적인 정보를 제공해주세요
5. 컨텍스트에 없는 내용은 "해당 정보를 찾을 수 없습니다"라고 답변하세요
6. 함수명이나 API명이 언급된 경우 구체적인 사용법을 설명해주세요
7. 스펙이나 규격 정보가 있다면 정확히 포함해주세요
8. 전체 맥락을 고려하여 답변해주세요

컨텍스트:
{context}

질문: {question}

답변:"""

prompt = PromptTemplate.from_template(prompt_template)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ 맥락 보존 RAG 체인이 생성되었습니다!")

In [ ]:
# 맥락 보존 RAG 체인 테스트
print("=== 맥락 보존 RAG 체인 테스트 ===")

test_questions = [
    "launchPurchaseFlow() 함수는 언제 사용되나요?",
    "PurchaseClient 초기화 방법을 알려주세요",
    "queryProductDetailAsync API 사용법을 설명해주세요",
    "IapResult와 PurchaseData의 차이점은?",
    "인앱결제 구매 프로세스를 단계별로 설명해주세요",
    "PNS(Push Notification Service)는 무엇인가요?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\\n📝 질문 {i}: {question}")
    print("-" * 60)
    
    try:
        answer = rag_chain.invoke(question)
        print(answer)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("=" * 80)

In [ ]:
# 검색 결과 분석 (맥락 보존)
def analyze_context_search_results(question: str, top_k: int = 5):
    """맥락 보존 검색 결과를 분석하여 상세 정보를 제공합니다."""
    print(f"🔍 질문: {question}")
    print(f"📊 상위 {top_k}개 검색 결과 분석\\n")
    
    # 검색 결과 가져오기
    docs = retriever.invoke(question)
    
    for i, doc in enumerate(docs[:top_k], 1):
        print(f"📄 결과 {i}:")
        print(f"   타입: {doc.metadata.get('type', 'unknown')}")
        print(f"   소스: {doc.metadata.get('source', 'unknown')}")
        print(f"   키워드: {doc.metadata.get('keywords', [])}")
        print(f"   코드 포함: {doc.metadata.get('has_code', False)}")
        print(f"   함수 포함: {doc.metadata.get('has_function', False)}")
        print(f"   분할 방법: {doc.metadata.get('split_method', 'unknown')}")
        print(f"   청크 크기: {len(doc.page_content)} 문자")
        print(f"   내용: {doc.page_content[:200]}...")
        print("   " + "-" * 50)
    
    print(f"\\n✅ 총 {len(docs)}개의 관련 문서를 찾았습니다.")

# 검색 결과 분석 테스트
analyze_context_search_results("launchPurchaseFlow", top_k=3)
print("\\n" + "=" * 80)
analyze_context_search_results("PurchaseClient", top_k=3)